We wish to find the Minimum Edit Distance (MED) between two strings. That is, given two strings, align them, and find the minimum operations from {Insert, Delete, Substitute} needed to get from the first string to the second string.
Then, we want to find the actual operations done in order to reach this MED, e.g "Insert 'A' at position 3".

<img src="./MinimumEditDistance1.jpg" />

We can try and achieve this goal using Dynamic Programming (DP) for optimal complexity as follows:
Define:
* String 1: $X$ of length $n$
* String 2: $Y$ of length $m$
* $D[i,j]$: Edit Distance between substrings $X[1 \rightarrow i]$ and $Y[1 \rightarrow j]$

Using "Bottom Up" approach, the MED between $X$ and $Y$ would be $D[n,m]$.

We assume that the distance between string of length 0 to a string of length k is k, since we need to insert k characters is order to create string 2.

The matrix initially looks like this:

<img src="./MinimumEditDistance2.jpg" />

In order to actually find the operation, we need to keep track of the operations, that is, create a "Backtrace":

<img src="./MinimumEditDistance3.jpg" />

Complexity:

* Time: O(n*m)
* Space: O(n*m)
* Backtrace: O(n+m)


In [1]:
# Py file:
"""
Author: Tal Daniel
Minimum Edit Distance with Backtrace
-----------------------------------

We wish to find the Minimum Edit Distance (MED) between two strings. That is,
given two strings, align them, and find the minimum operations from {Insert,
Delete, Substitute} needed to get from the first string to the second string.
Then, we want to find the actual operations done in order to reach this MED,
e.g "Insert 'A' at position 3".

We can try and achieve this goal using Dynamic Programming (DP) for optimal
complexity as follows: Define:
* String 1: $X$ of length $n$
* String 2: $Y$ of length $m$
* $D[i,j]$: Edit Distance between substrings $X[1 \rightarrow i]$ and $Y[1 \rightarrow j]$

Using "Bottom Up" approach, the MED between $X$ and $Y$ would be $D[n,m]$.

We assume that the distance between string of length 0 to a string of length k
is k, since we need to insert k characters is order to create string 2.  In
order to actually find the operation, we need to keep track of the operations,
that is, create a "Backtrace".


Complexity:

* Time: O(n*m)
* Space: O(n*m)
* Backtrace: O(n+m)

"""

'\nAuthor: Tal Daniel\nMinimum Edit Distance with Backtrace\n-----------------------------------\n\nWe wish to find the Minimum Edit Distance (MED) between two strings. That is,\ngiven two strings, align them, and find the minimum operations from {Insert,\nDelete, Substitute} needed to get from the first string to the second string.\nThen, we want to find the actual operations done in order to reach this MED,\ne.g "Insert \'A\' at position 3".\n\nWe can try and achieve this goal using Dynamic Programming (DP) for optimal\ncomplexity as follows: Define:\n* String 1: $X$ of length $n$\n* String 2: $Y$ of length $m$\n* $D[i,j]$: Edit Distance between substrings $X[1 \rightarrow i]$ and $Y[1 \rightarrow j]$\n\nUsing "Bottom Up" approach, the MED between $X$ and $Y$ would be $D[n,m]$.\n\nWe assume that the distance between string of length 0 to a string of length k\nis k, since we need to insert k characters is order to create string 2.  In\norder to actually find the operation, we need to 

In [2]:
# Imports:

import numpy as np
import string
import json
import csv
import itertools
import time
from word2keypress import distance, Keyboard
from ast import literal_eval

In [3]:
def find_med_backtrace(str1, str2):
    '''
    This function calculates the Minimum Edit Distance between 2 words using
    Dynamic Programming, and asserts the optimal transition path using backtracing.
    Input parameters: original word, target word
    Output: minimum edit distance, path
    Example: ('password', 'Passw0rd') -> 2.0, [('s', 'P', 0), ('s', '0', 5)]
    '''
    # Definitions:
    n = len(str1)
    m = len(str2)
    D = np.full((n + 1, m + 1), np.inf)
    op_arr_str = ["d", "i", "c", "s"]
    trace = np.full((n + 1, m + 1), None)
    for i in range(1 , n + 1):
        trace[i ,0] = (i - 1 ,0)
    for j in range(1 , m + 1):
        trace[0 ,j] = (0 ,j - 1)
    # Initialization:
    for i in range(n + 1):
        D[i,0] = i
    for j in range(m + 1):
        D[0,j] = j
    # Fill the matrices:
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            delete = D[i - 1, j] + 1
            insert = D[i, j-1] + 1
            if (str1[i - 1] == str2[j - 1]):
                sub = np.inf
                copy = D[i - 1, j - 1]
            else:
                sub = D[i - 1, j - 1] + 1
                copy = np.inf
            op_arr = [delete, insert, copy, sub]
            D[i ,j] = np.min(op_arr)
            op = np.argmin(op_arr)
            if (op == 0):
                # delete, go down
                trace[i,j] = (i-1, j)
            elif (op == 1):
                # insert, go left
                trace[i,j] = (i, j-1)
            else:
                # copy or subsitute, go diag
                trace[i,j] = (i-1, j-1)
#     print(trace)
    # Find the path of transitions:
    i = n
    j = m
    cursor = trace[i ,j]
    path = []
    while (cursor is not None):
        # 3 possible directions:
#         print(cursor)
        if (cursor[0] == i - 1 and cursor[1] == j - 1):
            # diagonal - sub or copy
            if (str1[cursor[0]] != str2[cursor[1]]):
                # substitute
                path.append(("s", str2[cursor[1]], cursor[0]))
            i = i - 1
            j = j - 1
        elif (cursor[0] == i and cursor[1] == j - 1):
            # go left - insert
            path.append(("i", str2[cursor[1]], cursor[0]))
            j = j - 1
        else:
            # (cursor[0] == i - 1 and cursor[1] == j )
            # go down - delete
            path.append(("d", None, cursor[0]))
            i = i - 1
        cursor = trace[cursor[0], cursor[1]]
    return D[n ,m], list(reversed(path))

In [4]:
# Minimum Edit Distance with Backtrace
def find_med_backtrace_kb(str1, str2):
    '''
    This function calculates the Minimum Edit Distance between 2 words using
    Dynamic Programming, and asserts the optimal transition path using backtracing.
    This version uses KeyPress representation.
    Input parameters: original word, target word
    Output: minimum edit distance, path
    Example:
    ('password', 'PASSword') -> 2.0 , [('i', '\x04', 0), ('i', '\x04', 4)]
    '''
    # Transform to keyboard representation:
    kb = Keyboard()
    str1 = kb.word_to_keyseq(str1)
    str2 = kb.word_to_keyseq(str2)
    # Definitions:
    n = len(str1)
    m = len(str2)
    D = np.full((n + 1, m + 1), np.inf)
    op_arr_str = ["d", "i", "c", "s"]
    trace = np.full((n + 1, m + 1), None)
    for i in range(1 , n + 1):
        trace[i ,0] = (i - 1 ,0)
    for j in range(1 , m + 1):
        trace[0 ,j] = (0 ,j - 1)
    # Initialization:
    for i in range(n + 1):
        D[i,0] = i
    for j in range(m + 1):
        D[0,j] = j
    # Fill the matrices:
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            delete = D[i - 1, j] + 1
            insert = D[i, j-1] + 1
            if (str1[i - 1] == str2[j - 1]):
                sub = np.inf
                copy = D[i - 1, j - 1]
            else:
                sub = D[i - 1, j - 1] + 1
                copy = np.inf
            op_arr = [delete, insert, copy, sub]
            D[i ,j] = np.min(op_arr)
            op = np.argmin(op_arr)
            if (op == 0):
                # delete, go down
                trace[i,j] = (i-1, j)
            elif (op == 1):
                # insert, go left
                trace[i,j] = (i, j-1)
            else:
                # copy or subsitute, go diag
                trace[i,j] = (i-1, j-1)
#     print(trace)
    # Find the path of transitions:
    i = n
    j = m
    cursor = trace[i ,j]
    path = []
    while (cursor is not None):
        # 3 possible directions:
#         print(cursor)
        if (cursor[0] == i - 1 and cursor[1] == j - 1):
            # diagonal - sub or copy
            if (str1[cursor[0]] != str2[cursor[1]]):
                # substitute
                path.append(("s", str2[cursor[1]], cursor[0]))
            i = i - 1
            j = j - 1
        elif (cursor[0] == i and cursor[1] == j - 1):
            # go left - insert
            path.append(("i", str2[cursor[1]], cursor[0]))
            j = j - 1
        else:
            # (cursor[0] == i - 1 and cursor[1] == j )
            # go down - delete
            path.append(("d", None, cursor[0]))
            i = i - 1
        cursor = trace[cursor[0], cursor[1]]
    return D[n ,m], list(reversed(path))

In [5]:
med, path = find_med_backtrace_kb('password', 'Passw0rd')
print(med)
print(path)

2.0
[('i', '\x03', 0), ('s', '0', 5)]


In [6]:
# Decoder - given a word and a path of transition, recover the final word:
def path2word(word, path):
    '''
    This function decodes the word in which the given path transitions the input word into.
    Input parameters: original word, transition path
    Output: decoded word
    '''
    if not path:
        return word
    final_word = []
    word_len = len(word)
    path_len = len(path)
    i = 0
    j = 0
    while (i < word_len or j < path_len):
        if (j < path_len and path[j][2] == i):
            if (path[j][0] == "s"):
                # substitute
                final_word.append(path[j][1])
                i += 1
                j += 1
            elif (path[j][0] == "d"):
                # delete
                i += 1
                j += 1
            else:
                # "i", insert
                final_word.append(path[j][1])
                j += 1
        else:
            final_word.append(word[i])
            i += 1
    return ''.join(final_word)

In [7]:
# Decoder - given a word and a path of transition, recover the final word: KEYPRESS Version
def path2word_kb(word, path):
    '''
    This function decodes the word in which the given path transitions the input word into.
    This is the KeyPress version, which handles the keyboard representations.
    Input parameters: original word, transition path
    Output: decoded word
    '''
    kb = Keyboard()
    word = kb.word_to_keyseq(word)
    if not path:
        return kb.keyseq_to_word(word)
    final_word = []
    word_len = len(word)
    path_len = len(path)
    i = 0
    j = 0
    while (i < word_len or j < path_len):
        if (j < path_len and path[j][2] == i):
            if (path[j][0] == "s"):
                # substitute
                final_word.append(path[j][1])
                i += 1
                j += 1
            elif (path[j][0] == "d"):
                # delete
                i += 1
                j += 1
            else:
                # "i", insert
                final_word.append(path[j][1])
                j += 1
        else:
            final_word.append(word[i])
            i += 1
    return (kb.keyseq_to_word(''.join(final_word)))

In [8]:
# Simple test:
pair = ("password", "P@SSword")
med, path = find_med_backtrace_kb(pair[0], pair[1])
print(med)
print(path)
decoded_word = path2word_kb(pair[0], path)
print(decoded_word)

4.0
[('i', '\x04', 0), ('s', '\x03', 1), ('i', '2', 2), ('i', '\x04', 4)]
P@SSword


In [9]:
def generate_transition_dict():
    '''
    Generate a dictionary of all possible paths in a JSON format
    Assumptions: words' max length is 30 chars and words are comprised of 98 available characters
    'd' - ('d', None, 0-30) -> 31 options
    's' - ('s', 0-95, 0-30) -> 98x31 = 3038 options
    'i' - ('i', 0-95, 0-30) -> 98x31 = 3038 options
    Size of table: 31 + 3038 + 3038 = 6107
    '''
    max_len = 31
    d_list = [('d', None, i) for i in range(max_len)]
    asci = list(string.ascii_letters)
    punc = list(string.punctuation)
    dig = list(string.digits)
    chars = asci + punc + dig + [" ", "\t", "\x03", "\x04"]
    s_list = [('s', c, i) for c in chars for i in range(max_len)]
    i_list = [('i', c, i) for c in chars for i in range(max_len)]

    transition_table = d_list + s_list + i_list
    transition_dict_2idx = {}
    transition_dict_2path = {}
    for i in range(len(transition_table)):
        transition_dict_2idx[str(transition_table[i])] = i
        transition_dict_2path[i] = str(transition_table[i])
    with open('trans_dict_2idx.json', 'w') as outfile:  
        json.dump(transition_dict_2idx, outfile)
    with open('trans_dict_2path.json', 'w') as outfile:  
        json.dump(transition_dict_2path, outfile)
    print("Transitions dictionary created as trans_dict_2idx.json & trans_dict_2path.json")
    '''
    Read: 
    if filename:
        with open(filename, 'r') as f:
            transition_dict = json.load(f)
    '''

In [10]:
generate_transition_dict()

Transitions dictionary created as trans_dict_2idx.json & trans_dict_2path.json


In [11]:
def path2idx(path, dictionary):
    '''
    This functions converts human-readable transition path to a
    dictionary-indices path (for future use in RNNs).
    Input parameters: human-readable path, dictionary
    Output: dictionary-indices path
    [('i', '\x04', 0), ('s', '\x03', 1), ('i', '2', 2), ('i', '\x04', 4)] ->
    [6076, 3008, 5737, 6080]
    '''
    idx_path = []
    for p in path:
        idx_path.append(dictionary[str(p)])
    return idx_path

In [12]:
def idx2path(path, dictionary):
    '''
    This functions converts dictionary-indices transition path to a
    human-readable path (for future use in RNNs).
    Input parameters: human-readable path, dictionary
    Output: dictionary-indices path
    [6076, 3008, 5737, 6080] ->
    [('i', '\x04', 0), ('s', '\x03', 1), ('i', '2', 2), ('i', '\x04', 4)]
    '''
    str_path = []
    for i in path:
        str_path.append(literal_eval(dictionary[str(i)]))
    return str_path

In [100]:
def idx2path_no_json(path, dictionary):
    '''
    This functions converts dictionary-indices transition path to a
    human-readable path (for future use in RNNs).
    Input parameters: human-readable path, dictionary
    Output: dictionary-indices path
    [6076, 3008, 5737, 6080] ->
    [('i', '\x04', 0), ('s', '\x03', 1), ('i', '2', 2), ('i', '\x04', 4)]
    '''
    str_path = []
    path = literal_eval(path)
    for i in path:
        str_path.append(dictionary[str(i)])
    return str_path

In [13]:
with open('trans_dict_2idx.json', 'r') as f:
    tran_dict = json.load(f)
idx_path = path2idx(path, tran_dict)
print(idx_path)

[6076, 3008, 5737, 6080]


In [14]:
with open('trans_dict_2path.json', 'r') as f:
    tran_dict = json.load(f)
str_path = idx2path(idx_path, tran_dict)
print(str_path)

[('i', '\x04', 0), ('s', '\x03', 1), ('i', '2', 2), ('i', '\x04', 4)]


In [15]:
def csv2pws_pairs(csv_fpath):
    '''
    Function to parse the csv file, such that every row is a list of username and a string of passwords list.
    Using itertools, find all the combinations of passwords, and generate an appropriate path.
    For every password and path, build the output password, and compare the result with the original pair.
    Input parameter: path to original dataset csv
    '''
    #     csv_fpath = './sample_username_list_tr.csv'
    pws_pairs = []
    with open(csv_fpath) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if (len(row) != 2):
                print("File format error!")
                break
            username = row[0]
            pws_string = row[1]
            pws_list = json.loads(pws_string)
            pws_pairs.extend(list(itertools.permutations(pws_list,2)))
    return pws_pairs

In [16]:
def csv2pws_pairs_gen(csv_fpath):
    '''
    Generator function to parse the csv file, such that every row is a list of username and a string of passwords list.
    Using itertools, find all the combinations of passwords, and generate an appropriate path.
    For every password and path, build the output password, and compare the result with the original pair.
    Input parameter: path to original dataset csv
    '''
    #     csv_fpath = './sample_username_list_tr.csv'
    #     pws_pairs = []
    with open(csv_fpath) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for i, row in enumerate(csv_reader):
            if (len(row) != 2):
                print("File format error @ line {}\n{!r}!".format(i, row))
                break
            username, pws_string = row
            pws_list = json.loads(pws_string)
            for p in itertools.permutations(pws_list, 2):
                yield p

In [88]:
def csv2pws_pairs_gen_no_json(csv_fpath):
    '''
    Generator function to parse the csv file, such that every row is a list of username and a string of passwords list.
    Using itertools, find all the combinations of passwords, and generate an appropriate path.
    For every password and path, build the output password, and compare the result with the original pair.
    Input parameter: path to original dataset csv
    '''
    #     csv_fpath = './sample_username_list_tr.csv'
    #     pws_pairs = []
    with open(csv_fpath) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for i, row in enumerate(csv_reader):
            if (len(row) != 2):
                print("File format error @ line {}\n{!r}!".format(i, row))
                break
            if not i:
                username, pws_string = "", "[]"
            else:
                username, pws_string = row
            pws_list = eval(pws_string)
            for p in itertools.permutations(pws_list, 2):
                yield p

In [17]:
def run_test(csv_fpath):
    '''
    This function tests the encoder-decoder functions, in order to make sure
    that for evey transition path from pass1 to pass2, the decoded password from pass1
    and the transition path is the same as pass2.
    '''
    start = time.clock()
    pws_pairs_gen = csv2pws_pairs_gen(csv_fpath)
    for i, pair in enumerate(pws_pairs_gen):
        med, path = find_med_backtrace_kb(pair[0], pair[1])
        decoded_word = path2word_kb(pair[0], path)
        if (decoded_word != pair[1]):
            print("Test failed on: {}".format(pair))
            print("Path chosen: {}".format(path))
            print("Decoded Password: {}".format(decoded_word))
    print("Testing done in {} seconds on a total of {} passwords pairs"
          .format(time.clock() - start, i))

In [18]:
# Testing
csv_fpath = './sample_username_list_tr.csv'
run_test(csv_fpath)

Testing done in 152.82311970842693 seconds on a total of 179657 passwords pairs


In [19]:
# Generate new dataset:
def csv2dataset(csv_fpath):
    '''
    This function generates the new dataset format from the original one.
    The new dataset is in the form: [pass1, pass2, human-readable transition path].
    Input parameter: path to original dataset csv
    '''
    kb = Keyboard()
    print("Started building dataset...")
    start = time.clock()
    dataset = []
    pws_pairs = csv2pws_pairs(csv_fpath)
    with open('trans_dataset.csv', 'w', newline='') as csvfile:
        for i, pair in enumerate(pws_pairs):
            med, path = find_med_backtrace_kb(pair[0], pair[1])
            decoded_word = path2word_kb(pair[0], path)
            str_path = [str(p) for p in path]
            if (decoded_word != pair[1]):
                print("Test failed on: {}".format(pair))
                print("Path chosen: {}".format(path))
                print("Decoded Password: {}".format(decoded_word))
            else:
                dataset.append((pair[0], pair[1], path))
            if (i % 50000 == 0):
                print("Progress: {}%".format((i / len(pws_pairs))*100.0))
                #  print(str_path)
            csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow([json.dumps(pair[0]), json.dumps(pair[1]), json.dumps(str_path)])
    print("Dataset created in {} seconds on a total of {} passwords pairs".format(time.clock() - start, len(pws_pairs)))
    print("New Dataset CSV file: trans_dataset.csv")
    return dataset

In [20]:
def csv2dataset_gen(csv_fpath):
    '''
    This (generator) function generates the new dataset format from the original one.
    The new dataset is in the form: [pass1, pass2, human-readable transition path].
    Input parameter: path to original dataset csv
    '''
    kb = Keyboard()
    print("Started building dataset...")
    start = time.clock()
    pairs_generator = csv2pws_pairs_gen(csv_fpath)
    with open('trans_dataset.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(
            csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL
        )
        for i, pair in enumerate(pairs_generator):
            med, path = find_med_backtrace_kb(pair[0], pair[1])
            decoded_word = path2word_kb(pair[0], path)
            str_path = [str(p) for p in path]
            if (decoded_word != pair[1]):
                print("Test failed on: {}".format(pair))
                print("Path chosen: {}".format(path))
                print("Decoded Password: {}".format(decoded_word))
            if (i % 50000 == 0):
                print("Progress: processed {} pairs so far".format(i))
            csv_writer.writerow([pair[0], pair[1], json.dumps(str_path)])
    print("Dataset created in {} seconds on a total of {} passwords pairs".format(time.clock() - start, i))
    print("New Dataset CSV file: trans_dataset.csv")

In [78]:
def csv2dataset_dict_gen(csv_fpath, dict_json):
    '''
    This (generator) function generates the new dataset format from the original one.
    The new dataset is in the form: [pass1, pass2, dictionary-indices transition path].
    Input parameter: path to original dataset csv, path to the json dictionary file
    '''
    kb = Keyboard()
    print("Started building dataset...")
    start = time.clock()
    pairs_generator = csv2pws_pairs_gen(csv_fpath)
    with open(dict_json, 'r') as f:
        trans_dict = json.load(f)
    with open('trans_dataset.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(
            csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL
        )
        for i, pair in enumerate(pairs_generator):
            if not i:
                # skip first line
                continue
            if (len(pair[0]) > 30 or len(pair[1]) > 30):
                continue
            med, path = find_med_backtrace_kb(pair[0], pair[1])
            decoded_word = path2word_kb(pair[0], path)
            str_path = [str(p) for p in path]
            pws_indices = path2idx(str_path, trans_dict)
            if (decoded_word != pair[1]):
                print("Test failed on: {}".format(pair))
                print("Path chosen: {}".format(path))
                print("Decoded Password: {}".format(decoded_word))
            if (i % 50000 == 0):
                print("Progress: processed {} pairs so far".format(i))
            csv_writer.writerow([
                pair[0], pair[1], json.dumps(pws_indices)
            ])
    print("Dataset created in {} seconds on a total of {} passwords pairs".format(time.clock() - start, i))
    print("New Dataset CSV file: trans_dataset.csv")

In [123]:
def csv2dataset_dict_gen_no_json(csv_fpath, dict_json):
    '''
    This (generator) function generates the new dataset format from the original one.
    The new dataset is in the form: [pass1, pass2, dictionary-indices transition path].
    Input parameter: path to original dataset csv, path to the json dictionary file
    '''
    kb = Keyboard()
    print("Started building dataset...")
    start = time.clock()
    pairs_generator = csv2pws_pairs_gen_no_json(csv_fpath)
    with open(dict_json, 'r') as f:
        trans_dict = json.load(f)
    with open('trans_dataset.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(
            csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL
        )
        for i, pair in enumerate(pairs_generator):
            if not i:
                # skip first line
                continue
            if (len(pair[0]) > 30 or len(pair[1]) > 30):
                continue
            med, path = find_med_backtrace_kb(pair[0], pair[1])
            skip = False
            for p in path:
                if p[2] > 30:
                    skip = True
                if not trans_dict.get(str(p)):
                    # Key not in dictionary
                    skip = True
            if skip:
                continue
            decoded_word = path2word_kb(pair[0], path)
            str_path = [str(p) for p in path]
            pws_indices = path2idx(str_path, trans_dict)
            if (decoded_word != pair[1]):
                print("Test failed on: {}".format(pair))
                print("Path chosen: {}".format(path))
                print("Decoded Password: {}".format(decoded_word))
            if (i % 50000 == 0):
                print("Progress: processed {} pairs so far".format(i))
            csv_writer.writerow([
                pair[0], pair[1], json.dumps(pws_indices)
            ])
    print("Dataset created in {} seconds on a total of {} passwords pairs".format(time.clock() - start, i))
    print("New Dataset CSV file: trans_dataset.csv")

In [89]:
csv_fpath = './cleaned_user_pass_tr_50.csv'
dict2idx_json = 'trans_dict_2idx.json'
dict2path_json = 'trans_dict_2path.json'
# csv2dataset_gen(csv_fpath)
csv2dataset_dict_gen_no_json(csv_fpath, dict2idx_json)

Started building dataset...
Dataset created in 3.17625931085513 seconds on a total of 3707 passwords pairs
New Dataset CSV file: trans_dataset.csv


In [23]:
def csv2trans_dataset(dataset_csv, dict_json):
    '''
    This function reads and processes the new dataset from a csv file,
    and parses the human-readable transition path into a dictionary indices transition path,
    using an input dictionary file.
    A sample is now a tuple in the form of:
    [pass1, pass2, human-readable transition path, dictionary indices transition path]
    Input parameters: path to the dataset csv file, path to the json dictionary file.
    Output: list of tuples in the mentioned form.
    '''
    with open(dict_json, 'r') as f:
        trans_dict = json.load(f)
    samples = []  # (pass1, pass2, path, idx_path)
    with open(dataset_csv) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if (len(row) != 3):
                print("File format error!")
                break
            pass_1 = row[0]
            pass_2 = row[1]
            pws_str = row[2]
            pws_list = json.loads(pws_str)
            pws_indices = path2idx(pws_list, trans_dict)
            samples.append((pass_1, pass_2, pws_list, pws_indices))
    return samples

In [24]:
def csv2trans_dataset_gen(dataset_csv, dict_json):
    '''
    This (generator) function reads and processes the new dataset from a csv file,
    and parses the human-readable transition path into a dictionary indices transition path,
    using an input dictionary file.
    A sample is now a tuple in the form of:
    [pass1, pass2, human-readable transition path, dictionary indices transition path]
    The csv file is in the form:
    [pass1, pass2, human-readable transition path]
    Input parameters: path to the dataset csv file, path to the json dictionary file.
    Output: yields a tuple of the mentiond form
    '''
    with open(dict_json, 'r') as f:
        trans_dict = json.load(f)
    with open(dataset_csv) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if (len(row) != 3):
                print("File format error!")
                break
            pass_1, pass_2, pws_str = row
            pws_list = json.loads(pws_str)
            pws_indices = path2idx(pws_list, trans_dict)
            yield (pass_1, pass_2, pws_list, pws_indices)

In [25]:
def csv2trans_dataset_dict_gen(dataset_csv, dict2path_json):
    '''
    This (generator) function reads and processes the new dataset from a csv file,
    and parses the dictionary indices transition path into a human-readable path.
    using an input dictionary file.
    A sample is now a tuple in the form of:
    [pass1, pass2, human-readable transition path, dictionary indices transition path]
    The csv file is in the form:
    [pass1, pass2, human-readable transition path]
    Input parameters: path to the dataset csv file, path to the json dictionary file.
    Output: yields a tuple of the mentiond form
    '''
    with open(dict2path_json, 'r') as f:
        trans_dict = json.load(f)
    with open(dataset_csv) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if (len(row) != 3):
                print("File format error!")
                break
            pass_1, pass_2, pws_indices = row
            pws_list = idx2path(pws_indices, trans_dict)
            yield (pass_1, pass_2, pws_list, pws_indices)

In [92]:
def csv2trans_dataset_dict_gen_no_json(dataset_csv, dict2path_json):
    '''
    This (generator) function reads and processes the new dataset from a csv file,
    and parses the dictionary indices transition path into a human-readable path.
    using an input dictionary file.
    A sample is now a tuple in the form of:
    [pass1, pass2, human-readable transition path, dictionary indices transition path]
    The csv file is in the form:
    [pass1, pass2, human-readable transition path]
    Input parameters: path to the dataset csv file, path to the json dictionary file.
    Output: yields a tuple of the mentiond form
    '''
    with open(dict2path_json, 'r') as f:
        trans_dict = json.load(f)
    with open(dataset_csv) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if (len(row) != 3):
                print("File format error!")
                break
            pass_1, pass_2, pws_indices = row
            pws_list = idx2path_no_json(pws_indices, trans_dict)
            yield (pass_1, pass_2, pws_list, pws_indices)

In [101]:
# Usage example:

dataset_csv = './trans_dataset.csv'
dict_json = 'trans_dict_2path.json'
# samples = csv2trans_dataset(dataset_csv, dict_json)

# # Using an array in memory:
# print("Password 1: {}, Password 2: {}".format(samples[4][0], samples[4][1]))
# print("Readable Transition Path: {}".format(samples[4][2]))
# print("Sequential Transition Path: {}".format(samples[4][3]))

# Using the generator:
samples_generator = csv2trans_dataset_dict_gen_no_json(dataset_csv, dict_json)
for i, sample in enumerate(samples_generator):
    if (i % 50000 == 0):
        print(sample)

('mama20062010', '19101970', ["('s', '1', 0)", "('s', '9', 1)", "('s', '1', 2)", "('d', None, 3)", "('d', None, 4)", "('s', '1', 6)", "('s', '9', 7)", "('s', '7', 8)", "('d', None, 10)", "('d', None, 11)"], '[2666, 2915, 2668, 3, 4, 2672, 2921, 2860, 10, 11]')


# Steps
## When running on the server

1. Generate dictionaries using: generate_transition_dict() [creates 2 dictionaries: path2idx, idx2path]
2. Create the new dataset using: csv2dataset_dict_gen(csv_fpath, dict2idx_json)
3. In order to generate samples: samples_generator = csv2trans_dataset_dict_gen(dataset_csv, dict2path_json)
4. Samples are in the form: [pass1, pass2, human-readable transition path, dictionary indices transition path], take what cells you need for training/testing

In [122]:
dict_json = 'trans_dict_2idx.json'
with open(dict_json, 'r') as f:
    trans_dict = json.load(f)
p = ('s', '\x0e', 7)
if not trans_dict.get(str(p)):
    # Key not in dictionary
    print("Key Not Found")

Key Not Found


In [126]:
len(string.digits)

10